In [1]:
import dill
import json
import os
import numpy as np

In [2]:
# main_dir = '../models/20210816_230313_perc_ubuntu_cog/'
main_dir = '../models/20211101_190014'
output_dir = f'{main_dir}/ML_obj_hyperparams'

In [3]:
with open(os.path.join(main_dir, 'ML_obj.p'), 'rb') as f:
    ml_obj = dill.load(f)

In [4]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [7]:
ml_obj['lr']['nn'].__dict__

{'param_distributions': {'hidden_layer_sizes': [(200,),
   (150,),
   100,
   (150, 100, 50),
   (100, 50, 25)],
  'activation': ['relu'],
  'solver': ['adam'],
  'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5],
  'learning_rate': ['constant', 'adaptive'],
  'beta_1': [0, 0.001, 0.01, 0.1, 0.3, 0.5, 0.9],
  'beta_2': [0, 0.001, 0.01, 0.1, 0.3, 0.5, 0.9]},
 'n_iter': 1,
 'random_state': None,
 'scoring': 'balanced_accuracy',
 'estimator': MLPClassifier(max_iter=1000000000),
 'n_jobs': -1,
 'iid': 'deprecated',
 'refit': True,
 'cv': 5,
 'verbose': 3,
 'pre_dispatch': '2*n_jobs',
 'error_score': nan,
 'return_train_score': False,
 'multimetric_': False,
 'best_index_': 0,
 'best_score_': 0.7942800944829342,
 'best_params_': {'solver': 'adam',
  'learning_rate': 'adaptive',
  'hidden_layer_sizes': (150,),
  'beta_2': 0.9,
  'beta_1': 0.9,
  'alpha': 0.05,
  'activation': 'relu'},
 'best_estimator_': MLPClassifier(alpha=0.05, beta_2=0.9, hidden_layer_sizes=(150,),
               learning_ra

In [9]:
def mine_cv_results(cv_results_dict, cv):
    interesting_keys = [
        'mean_test_score',
        'std_test_score'
    ]
    split_keys = [f'split{i}_test_score' for i in range(cv)]
    intersting_dict = {}
    for key in interesting_keys:
        intersting_dict[key] = cv_results_dict[key]
    for key in split_keys:
        intersting_dict[key] = cv_results_dict[key]
    return intersting_dict


In [21]:
for rfe_key in ml_obj:
    ml_dict = ml_obj[rfe_key]
    for clc_name, clc in ml_dict.items():
        hyper_dict = clc.best_estimator_.__dict__
        modified_dict = {}
        best_params = clc.best_params_
        best_idx = clc.best_index_
        best_score = clc.best_score_
        cv_results = clc.cv_results_
        cv_results = mine_cv_results(cv_results, clc.cv)
        tobedelted = (False,None)
        for key, val in hyper_dict.items():
            if key in ['_Booster', '_le', 'base_estimator_','estimators_','base_estimator',
                       '_tree','_y','_fit_X','loss_','init_','_rng']:
                tobedeleted= (True, key)
                print(tobedeleted)
            elif isinstance(val, np.ndarray):
                modified_dict[key] = val.tolist()
            elif isinstance(val, np.int32):
                modified_dict[key] = int(val)
            elif isinstance(val, np.int64):
                modified_dict[key] = int(val)
            elif key in ['_label_binarizer', '_random_state','_optimizer']:
                modified_dict[key] = str(val)
            
            elif (key == 'coefs_') or (key == 'intercepts_'):
                updated_v = []
                for v in val:
                    updated_v.append(v.tolist())
                modified_dict[key] = updated_v

            else:
                modified_dict[key] = val
        

        for key, val in cv_results.items():
            if key in ['_Booster', '_le', 'base_estimator_','estimators_','base_estimator',
                       '_tree','_y','_fit_X','loss_','init_','_rng']:
                tobedeleted= (True, key)
                print(tobedeleted)
            elif isinstance(val, np.ndarray):
                modified_dict[key] = val.tolist()
            elif isinstance(val, np.int32):
                modified_dict[key] = int(val)
            elif isinstance(val, np.int64):
                modified_dict[key] = int(val)
            elif key in ['_label_binarizer', '_random_state','_optimizer']:
                modified_dict[key] = str(val)
            
            elif (key == 'coefs_') or (key == 'intercepts_'):
                updated_v = []
                for v in val:
                    updated_v.append(v.tolist())
                modified_dict[key] = updated_v

            else:
                modified_dict[key] = val
        params_dict = {}
        for key, val in best_params.items():
            if key in ['_Booster', '_le', 'base_estimator_','estimators_','base_estimator',
                       '_tree','_y','_fit_X','loss_','init_','_rng']:
                tobedeleted= (True, key)
                print(tobedeleted)
            elif isinstance(val, np.ndarray):
                params_dict[key] = val.tolist()
            elif isinstance(val, np.int32):
                params_dict[key] = int(val)
            elif isinstance(val, np.int64):
                params_dict[key] = int(val)
            elif key in ['_label_binarizer', '_random_state','_optimizer']:
                params_dict[key] = str(val)
            
            elif (key == 'coefs_') or (key == 'intercepts_'):
                updated_v = []
                for v in val:
                    updated_v.append(v.tolist())
                params_dict[key] = updated_v

            else:
                params_dict[key] = val
        
        modified_dict['best_params'] = params_dict

        if tobedelted[0]:
            del hyper_dict[tobedeleted[1]]
        
        with open(os.path.join(output_dir, f'{rfe_key}_{clc_name}.json'), 'w') as f:
            try:
                json.dump(modified_dict, f)
            except Exception:
                print(modified_dict)

(True, 'loss_')


In [11]:
str(modified_dict['_optimizer'])

'<sklearn.neural_network._stochastic_optimizers.AdamOptimizer object at 0x0000023BE2586A88>'

In [19]:
best_params


{'solver': 'adam',
 'learning_rate': 'adaptive',
 'hidden_layer_sizes': (150,),
 'beta_2': 0.9,
 'beta_1': 0.9,
 'alpha': 0.05,
 'activation': 'relu'}